# Chapter 3 — Prompt Modes: System vs Role vs Instructions vs Style

This notebook teaches how different *prompt modes* shape an LLM’s output and gives hands-on exercises. We will cover:

1. **System Prompt** — the default behavior / ground rules.
2. **Role Prompt** — the persona or hat worn for a given task.
3. **Instructions** — the specific command for the task at hand.
4. **Style** — tone and formatting.
5. **Mixed Frames** — combining modes for precise control.

> **Learning Goal**: By the end, you can design prompts that reliably change tone, depth, and format and you’ll have mini-exercises to practice each concept.


In [ ]:
# Run environment setup
def pretty_print_response():
    return
def get_completion():
    return
%run 00_Tutorial_How-To.ipynb

roles_config = {
    "Tour Guide": {
        "instruction": "Explain the Eiffel Tower to first-time visitors. Keep it practical.",
        "prompt": "You are a Tour Guide. Answer as a friendly local expert.",
        "temperature": 0.8
    },
    "Engineer": {
        "instruction": "Explain the Eiffel Tower focusing on materials and forces.",
        "prompt": "You are an Engineer. Be precise, structured, and quantitative.",
        "temperature": 0.2
    },
    "Journalist": {
        "instruction": "Explain the Eiffel Tower as a journalist would.",
        "prompt": "You are a Journalist. Be factual, balanced, and engaging.",
        "temperature": 0.55
    },
    "Educator": {
        "instruction": "What is the Eiffel Tower?",
        "prompt": "You are a historian specializing in 19th-century Europe. Always answer in formal academic English.",
        "temperature": 0.4
    }
}

## 1. What is a Prompt?
A **prompt** is the input you give an LLM. It can include background data, constraints, and explicit instructions. Think of prompt *modes* as dials:
- **System**: sets the default rules for the session.
- **Role**: tells the AI which persona to adopt.
- **Instructions**: the actual task to perform now.
- **Style**: how to sound and format the output.

## 2. System Prompt

**Definition**: The *system prompt* is the foundational directive that shapes default behavior for the model. It sets the ground rules and context for all responses that follow.


**How the prompt is sent to Ollama:**
When interacting with Ollama, prompts are structured as a sequence of roles and their content. This helps the model understand who is speaking and what context to use.


**Example prompt structure:**
```
=== Sending to Ollama ===
1. role=system
   content=You are a historian specializing in 19th-century Europe. Always answer in formal academic English.

2. role=user
   content=What is the Eiffel Tower?
=========================
```
The `system` role sets the default behavior, while the `user` role provides the question or task. This format ensures the model knows how to respond and what style or expertise to use.


**Why it matters:**
A well-crafted system prompt creates a persistent baseline for the session, guiding the model's tone, depth, and style unless later overridden by more specific instructions.

In [ ]:
# System prompt example using Educator role from roles_config
educator = roles_config["Educator"]
system_prompt = educator["prompt"]
prompt = educator["instruction"]
pretty_print_response(get_completion(prompt_or_messages=prompt, system_prompt=system_prompt))

## 3. Role Prompt

**Definition**: A *role prompt* lets you assign a temporary persona or perspective to the model for a specific task. This helps tailor the response to a particular audience or context.


**How role prompts are sent to Ollama:**
Prompts are structured as a sequence of roles and their content, making it clear who is speaking and what their focus should be.


**Example prompt structure:**
```
=== Sending to Ollama ===
1. role=role
   content=Tour Guide

2. role=user
   content=Explain the Eiffel Tower to first-time visitors. Keep it practical.
=========================
```
You can swap the role to "Engineer" or "Journalist" and adjust the instructions to change the focus and style of the response.


**Why it matters:**
Role prompts allow you to quickly change the model's perspective, expertise, or tone for a given task, without altering the overall session baseline. This makes it easy to get answers tailored to different audiences or needs.

In [ ]:
for role, config in roles_config.items():
    role_prompt = config["prompt"]
    instruction = config["instruction"]
    temp = config["temperature"]

    user_prompt = f"{role_prompt} {instruction}"
    pretty_print_response(get_completion(user_prompt, system_prompt="", temperature=temp))

## 4. Instructions
**Definition**: Direct commands describing *what to do now*. Instructions can override the default/system behavior set by the system prompt.

**Example in code below:**
The code below demonstrates how a formal academic system prompt can be overridden by a user instruction that asks for a casual explanation. This shows how instructions can change the model's output, even when a system prompt is present.

In [ ]:
# Example: Show system prompt being overridden by user instruction
system_prompt = "You are a formal academic historian. Always answer in formal academic English."
user_prompt = "Explain the Eiffel Tower as if you are talking to a child."
pretty_print_response(get_completion(user_prompt, system_prompt=system_prompt))

## 5. Style
**Definition**: Style controls the tone, mood, and formatting of the model's response (e.g., Academic, Storytelling, Concise, Playful).

**What we try to do:**
In the code below, we experiment with different styles to see how the model adapts its language, structure, and presentation. By specifying a style in plain text (e.g., "Style: Academic"), we can make the same instruction produce very different outputs—ranging from formal and detailed to casual and narrative.

**Examples:**
```
Style: Academic
Instructions: Explain the Eiffel Tower.
```
```
Style: Storytelling
Instructions: Explain the Eiffel Tower.
```

**Why it matters:**
Style determines how readable, engaging, and audience-appropriate the output is. It lets you tailor the model's response for different contexts and needs.

In [ ]:
# Style examples
styles = ["Academic", "Storytelling", "Concise"]
for style in styles:
    prompt = f"Style: {style}\nInstructions: Explain the Eiffel Tower."
    pretty_print_response(get_completion(prompt))

## 6. Mixed Frames (Combining Modes)
You can mix all four modes to precisely shape the output.

**Should you set a system prompt?**
Yes! When combining modes, it's best practice to set a system prompt to establish the default behavior and ground rules. The system prompt acts as a baseline, while role, style, instructions, and data further refine or override the output for each task.


**Override Example:**
A system prompt sets the default behavior, but a specific user instruction or role prompt can override it. For instance, set a formal academic system prompt, then provide a user instruction that asks for a casual explanation. You can also add data to the instructions, such as the location of the Eiffel Tower.

**Example (Plain text, as used in code):**
```
System: You are a precise, cite-when-appropriate science communicator.
Role: Engineer
Style: Concise
Instructions: Explain what it is and where it is in two bullets.
```

In [ ]:
# Mixed frames example
system = "You are a precise, cite-when-appropriate science communicator."
role = "Engineer"
style = "Concise"
instructions = "The Eiffel Tower is in Paris. Explain what it is and where it is in two bullets."
prompt = f"System: {system}\nRole: {role}\nStyle: {style}\nInstructions: {instructions}"
pretty_print_response(get_completion(prompt))

## 7. Quick Comparison Table

| Mode | Scope | Persistence | Primary Use |
|---|---|---|---|
| **System** | Global defaults | High | Set baseline behavior for the session |
| **Role** | Persona for a task | Low/Task-level | Change perspective/voice |
| **Instructions** | Immediate task | Immediate | Specify exact output and constraints |
| **Style** | Tone/formatting | Low/Task-level | Control how it reads/looks |

**Key Takeaways**
- System defines the playing field. Role, Instructions, and Style shape the specific play.
- You can override defaults by being explicit in instructions.
- Use mixed frames when you need both precision in *what* to say and *how* to say it.